In [2]:
import TimeTagger as TT

# set up swabian configs

In [ ]:
time_tagger = TT.createTimeTagger()

# file writer

In [ ]:
filename = ""
chanels = []
duration = 0 # in ps

In [ ]:
file_writer = TT.FileWriter(time_tagger, filename, channels)

In [ ]:
file_writer.startFor(duration)

# flim measurement

In [ ]:
#if used file writer
time_tagger.reset()

In [ ]:
laser = 1
frame = 3 
line = 2
photon = -4

n_bins = 256
binwidth = 49 # 12.5ns/256
n_pixel = 256

pixel_rate = 200e3
pixel_time = 1 / pixel_rate
integr_time = 3e9 # Integration time of 3 ms in picoseconds

In [ ]:
# EventGenerator(tagger, trigger_channel, pattern, trigger_divider, stop_channel)
#virtual channel
pixel_pattern_start = np.linspace(0, n_pixel, num=n_pixel, endpoint=False, dtype=int) * (pixel_time * 1e12)
pixel_pattern_end = pixel_pattern_start + integr_time

pixel_start_chan = TT.EventGenerator(time_tagger, line, pixel_pattern_start)
pixel_end_chan= TT.EventGenerator(time_tagger, line, pixel_pattern_end)

In [ ]:
#ConstantFractionDiscriminator(tagger, channels, search_window)
#virtual channel
photon_cfd_chan = TT.ConstantFractionDiscriminator(time_tagger, (photon,), 10*1000)

In [ ]:
#DelayedChannel(tagger, input_channel, delay)
#virtual channel
delayed_laser_chan = TT.DelayedChannel(time_tagger, laser, 11.5e3)

In [ ]:
#possibly don't need frame
flim = TT.Flim(time_tagger, start_channel=delayed_laser_chan.getChannel(), click_channel=photon_cfd_chan.getChannels()[0], pixel_begin_channel=pixel_start_chan.getChannel(), 
            n_pixels=n_pixel*n_pixel, n_bins=n_bins, binwidth=binwidth, pixel_end_channel=pixel_end_chan.getChannel(), frame_begin_channel=frame)

In [ ]:
flim.startFor(duration)

In [ ]:
flim_3d = flim.getSummedFrames().reshape(256,256,256)

In [ ]:
flim_mean = flim_2d.sum(2)

In [ ]:
figure(figsize=[7,7])
imshow(flim_mean)
colorbar()